# Expected Loss

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
# from sklearn.feature_selection import f_classif
# import scipy.stats as sm
from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression , LinearRegression
from sklearn.metrics import roc_auc_score,roc_curve, r2_score, mean_squared_error
import pickle
warnings.simplefilter('ignore')

# loading all saved model 

In [2]:
model_lgd_stage1_logistic_reg=pickle.load(open('model_logistic_stage_1_LGD.sav','rb'))
model_lgd_stage2_linear_reg=pickle.load(open('linearRegModel_stage2_LGD.sav','rb'))
model_ead_linear_reg=pickle.load(open('EAD_Model_linearReg.sav','rb'))
model_pd_logistic_reg=pickle.load(open('PD_reg_model.sav','rb'))

In [3]:
loan_data_preprocessed_LGD_EAD_backup=pd.read_csv('loan_data_2007_2014_preprocessed.csv')

In [4]:
loan_data_preprocessed_LGD_EAD=loan_data_preprocessed_LGD_EAD_backup.copy()

In [5]:
loan_data_preprocessed_LGD_EAD.shape

(466285, 209)

In [6]:
loan_data_preprocessed_LGD_EAD.head()

,Unnamed: 0,Unnamed: 0.1,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,...,addr_state:UT,addr_state:VA,addr_state:VT,addr_state:WA,addr_state:WI,addr_state:WV,addr_state:WY,initial_list_status:f,initial_list_status:w,good_bad
0,0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,...,0,0,0,0,0,0,0,1,0,1
1,1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,...,0,0,0,0,0,0,0,1,0,0
2,2,2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,...,0,0,0,0,0,0,0,1,0,1
3,3,3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,...,0,0,0,0,0,0,0,1,0,1
4,4,4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,...,0,0,0,0,0,0,0,1,0,1


In [7]:
pd.options.display.max_rows=None

# data preprocessing

In [8]:
loan_data_preprocessed_LGD_EAD.isnull().sum()

Unnamed: 0                                                              0
Unnamed: 0.1                                                            0
id                                                                      0
member_id                                                               0
loan_amnt                                                               0
funded_amnt                                                             0
funded_amnt_inv                                                         0
term                                                                    0
int_rate                                                                0
installment                                                             0
grade                                                                   0
sub_grade                                                               0
emp_title                                                           27588
emp_length                            

In [9]:
loan_data_preprocessed_LGD_EAD['mths_since_last_delinq'].fillna(0,inplace=True)
loan_data_preprocessed_LGD_EAD['mths_since_last_record'].fillna(0,inplace=True)


In [10]:
# preparing the inputs
feature_all=[
'grade:A',
'grade:B', 
'grade:C',
'grade:D',
'grade:E',
'grade:F',
'grade:G',

'home_ownership:ANY', 'home_ownership:MORTGAGE',
'home_ownership:NONE', 'home_ownership:OTHER',
'home_ownership:OWN', 'home_ownership:RENT',

'verification_status:Not Verified',
'verification_status:Source Verified',
'verification_status:Verified',

 'purpose:car', 'purpose:credit_card', 'purpose:debt_consolidation',
       'purpose:educational', 'purpose:home_improvement', 'purpose:house',
       'purpose:major_purchase', 'purpose:medical', 'purpose:moving',
       'purpose:other', 'purpose:renewable_energy',
       'purpose:small_business', 'purpose:vacation', 'purpose:wedding',

'initial_list_status:f',
'initial_list_status:w',

'term_int',
'emp_length_int',
'Mths_since_issue_d',
'Mths_since_earliest_cr_line',
'funded_amnt',
'int_rate',
'installment',
'annual_inc',
'dti',
'delinq_2yrs',
'inq_last_6mths',
'mths_since_last_delinq',
'mths_since_last_record',
'open_acc',
'pub_rec',
'total_acc',
'acc_now_delinq',
'total_rev_hi_lim'
]

In [11]:
feature_reference=[
'grade:G',
'home_ownership:RENT',
'verification_status:Verified',
'purpose:credit_card',
'initial_list_status:f',
]

In [12]:
loan_data_preprocessed_LGD_EAD=loan_data_preprocessed_LGD_EAD[feature_all].drop(columns=feature_reference)
loan_data_preprocessed_LGD_EAD.shape

(466285, 45)

In [13]:
df_ExpectedLoss=pd.DataFrame()
df_ExpectedLoss['rcvry_rate_stage_1']=model_lgd_stage1_logistic_reg.predict(loan_data_preprocessed_LGD_EAD)
df_ExpectedLoss['rcvry_rate_stage_2']=model_lgd_stage2_linear_reg.predict(loan_data_preprocessed_LGD_EAD)
df_ExpectedLoss['recovery_rate_final']=df_ExpectedLoss['rcvry_rate_stage_1']*df_ExpectedLoss['rcvry_rate_stage_2']

In [14]:
df_ExpectedLoss.describe()

,rcvry_rate_stage_1,rcvry_rate_stage_2,recovery_rate_final
count,466285.000000,466285.000000,466285.000000
mean,0.601409,0.122789,0.065585
std,0.489609,0.030089,0.057613
min,0.000000,-0.019925,-0.019925
25%,0.000000,0.106160,0.000000
50%,1.000000,0.126237,0.083347
75%,1.000000,0.143158,0.117437
max,1.000000,2.389546,0.631158


In [15]:
df_ExpectedLoss['recovery_rate_final']=np.where(df_ExpectedLoss['recovery_rate_final']<0,0,df_ExpectedLoss['recovery_rate_final'])
df_ExpectedLoss['recovery_rate_final']=np.where(df_ExpectedLoss['recovery_rate_final']>1,1,df_ExpectedLoss['recovery_rate_final'])

In [16]:
df_ExpectedLoss['LGD']=1-df_ExpectedLoss['recovery_rate_final']

In [17]:
df_ExpectedLoss.head()

,rcvry_rate_stage_1,rcvry_rate_stage_2,recovery_rate_final,LGD
0,1,0.086363,0.086363,0.913637
1,1,0.084464,0.084464,0.915536
2,1,0.080391,0.080391,0.919609
3,1,0.095262,0.095262,0.904738
4,1,0.088898,0.088898,0.911102


# calculating credit conversion factor

In [18]:
df_ExpectedLoss['CCF']=model_ead_linear_reg.predict(loan_data_preprocessed_LGD_EAD)
df_ExpectedLoss['CCF'].describe()

count    466285.000000
mean          0.737596
std           0.101871
min           0.292350
25%           0.671009
50%           0.730697
75%           0.803447
max           2.851196
Name: CCF, dtype: float64

In [19]:
df_ExpectedLoss['CCF']=np.where(df_ExpectedLoss['CCF']<0,0,df_ExpectedLoss['CCF'])
df_ExpectedLoss['CCF']=np.where(df_ExpectedLoss['CCF']>1,1,df_ExpectedLoss['CCF'])

In [20]:
df_ExpectedLoss['CCF'].describe()

count    466285.000000
mean          0.737579
std           0.101754
min           0.292350
25%           0.671009
50%           0.730697
75%           0.803447
max           1.000000
Name: CCF, dtype: float64

In [21]:
df_ExpectedLoss['EAD']=df_ExpectedLoss['CCF']*loan_data_preprocessed_LGD_EAD['funded_amnt']
df_ExpectedLoss['EAD'].describe()

count    466285.000000
mean      10814.852575
std        6935.165205
min         190.367768
25%        5495.036632
50%        9208.455337
75%       14692.882979
max       35000.000000
Name: EAD, dtype: float64

# calculating probability of default

In [22]:
loan_data_Preprocessed_PD_train=pd.read_csv('preprocessed_inputs_train.csv')
loan_data_Preprocessed_PD_test=pd.read_csv('preprocessed_inputs_test.csv')


In [23]:
loan_data_Preprocessed_PD_train.set_index(loan_data_Preprocessed_PD_train.columns.values[0],inplace=True)
loan_data_Preprocessed_PD_test.set_index(loan_data_Preprocessed_PD_test.columns.values[0],inplace=True)

In [24]:
selected_category_name=[
'grade:A',
'grade:B', 
'grade:C',
'grade:D',
'grade:E',
'grade:F',
'grade:G',

'home_ownership:MORTGAGE',
'home_ownership:OWN',
'home_ownership:OTHER_NONE_ANY_RENT',

'addr_state:NY',
'addr_state:CA',
'addr_state:TX',
'addr_state:ND',
'addr_state:NE_IA_NV_FL_HI_AL_ND',
'addr_state:WV_NH_WY_DC_ME_ID',
'addr_state:VA_NM',
'addr_state:OK_TN_MO_LA_MD_NC',
'addr_state:UT_KY_AZ_NJ',
'addr_state:AR_MI_PA_OH_MN',
'addr_state:RI_MA_DE_SD_IN',
'addr_state:GA_WA_OR',
'addr_state:WI_MT',
'addr_state:IL_CT',
'addr_state:KS_SC_CO_VT_AK_MS',

'verification_status:Not Verified',
'verification_status:Source Verified',
'verification_status:Verified',


'purpose:sml_busi_educational',
'purpose:mving_house_oths_renew_engy_medical',
'purpose:wedng_vction_debt_consldtn_wedding',
'purpose:credit_card',
'purpose:home_improvement',
'purpose:major_purchase',

'initial_list_status:f',
'initial_list_status:w',

'term_int:36',
'term_int:60',

'emp_length_int:0',
'emp_length_int:1',
'emp_length_int:2_3_4',
'emp_length_int:5_6',
'emp_length_int:7_8_9',
'emp_length_int:10',

'Mths_since_earliest_cr_line:61_78',
'Mths_since_earliest_cr_line:79_110',
'Mths_since_earliest_cr_line:111_127',
'Mths_since_earliest_cr_line:128_159',
'Mths_since_earliest_cr_line:160_241',
'Mths_since_earliest_cr_line:242_257',
'Mths_since_earliest_cr_line:258_298',
'Mths_since_earliest_cr_line:299_338',
'Mths_since_earliest_cr_line:339_420',
'Mths_since_earliest_cr_line:421_550',
'Mths_since_earliest_cr_line:551_876',

'int_rate:<=7.897',
'int_rate:7.897_9.548',
'int_rate:9.548_12.025',
'int_rate:12.025_14.089',
'int_rate:14.089_15.740',
'int_rate:15.740_20.281',
'int_rate:>20.281',


'annual_inc:<=19027',
'annual_inc:19027_27529',
'annual_inc:27529_36158',
'annual_inc:36158_44723',
'annual_inc:44723_50434',
'annual_inc:50434_61854',
'annual_inc:61854_73275',
'annual_inc:73275_81841',
'annual_inc:81841_101827',
'annual_inc:101827_121813',
'annual_inc:121813_144655',
'annual_inc:>144655',

'mths_since_last_delinq:Missing',
'mths_since_last_delinq:0_3',
'mths_since_last_delinq:4_30',
'mths_since_last_delinq:31_56',
'mths_since_last_delinq:>=57',

'Mths_since_issue_d:<28',
'Mths_since_issue_d:29<=>31',
'Mths_since_issue_d:32<=>34',
'Mths_since_issue_d:35<=>37',
'Mths_since_issue_d:38<=>40',
'Mths_since_issue_d:41<=>43',
'Mths_since_issue_d:44<=>49',
'Mths_since_issue_d:50<=>52',
'Mths_since_issue_d:53<=>61',
'Mths_since_issue_d:62<=>79',
'Mths_since_issue_d:80<=>97',
'Mths_since_issue_d:98<=',

'dti:<=1.6',
'dti:1.6<=>3.999',
'dti:3.999<=>8.798', 
'dti:8.798<=>11.997',
'dti:11.997<=>15.196',
'dti:15.196<=>16.796',
'dti:16.796<=>19.995',
'dti:19.995<=>23.994',
'dti:23.994<=>26.393',
'dti:26.393<=>31.992',
'dti:31.992<=>35.191', 
'dti:>35.191',

# 'delinq_2yrs:0',
# 'delinq_2yrs:1',
# 'delinq_2yrs:2',
# 'delinq_2yrs:3_4_5',
# 'delinq_2yrs:>6',

'inq_last_6mths:0',
'inq_last_6mths:1',
'inq_last_6mths:2',
'inq_last_6mths:3',
'inq_last_6mths:4_5_6',
'inq_last_6mths:>=7',

'open_acc:<=19',
'open_acc:20<=>39', 
'open_acc:>=40',

'pub_rec:0',
'pub_rec:1',
'pub_rec:2_3', 
'pub_rec:>=4',

'total_acc:<=8',
'total_acc:9<=>15',
'total_acc:16<=>23',
'total_acc:24<=>27',
'total_acc:28<=>45',

'acc_now_delinq:0', 
'acc_now_delinq:>0',

'total_rev_hi_lim:<=19760',
'total_rev_hi_lim:19760<=>39520',
'total_rev_hi_lim:39520<=>59280',
'total_rev_hi_lim:59280<=>79040', 
'total_rev_hi_lim:>79040',

'mths_since_last_record:missing',
'mths_since_last_record:<3',
'mths_since_last_record:3<=>31',
'mths_since_last_record:32<=>50',
'mths_since_last_record:51<=>72',
'mths_since_last_record:73<=>84',
'mths_since_last_record:85<=>96',
'mths_since_last_record:97<=>113',
'mths_since_last_record:>=114'
]

In [25]:
reference_col_to_remove=[
'grade:G',

'home_ownership:OTHER_NONE_ANY_RENT',

'addr_state:NE_IA_NV_FL_HI_AL_ND',

'verification_status:Verified',

'purpose:sml_busi_educational',

'initial_list_status:f',

'emp_length_int:0',

'Mths_since_earliest_cr_line:551_876',

'int_rate:>20.281',

'Mths_since_issue_d:98<=',

'term_int:60',

'dti:31.992<=>35.191',

# 'delinq_2yrs:>6',

'inq_last_6mths:>=7',

'open_acc:<=19',

'pub_rec:0',

'total_acc:<=8',

'acc_now_delinq:0',

'total_rev_hi_lim:<=19760',

'mths_since_last_record:<3'
]

In [26]:
loan_data_Preprocessed_PD_train.shape,loan_data_Preprocessed_PD_test.shape

((373028, 338), (93257, 329))

In [27]:
loan_data_Preprocessed_PD_train=loan_data_Preprocessed_PD_train[selected_category_name].drop(columns=reference_col_to_remove)
loan_data_Preprocessed_PD_test=loan_data_Preprocessed_PD_test[selected_category_name].drop(columns=reference_col_to_remove)

In [28]:
loan_data_Preprocessed_PD_train.shape,loan_data_Preprocessed_PD_test.shape

((373028, 118), (93257, 118))

In [29]:
loan_data_preprocessed_PD=pd.concat([loan_data_Preprocessed_PD_train,loan_data_Preprocessed_PD_test],axis=0)

In [30]:
loan_data_preprocessed_PD.shape,loan_data_preprocessed_LGD_EAD.shape

((466285, 118), (466285, 45))

In [31]:
loan_data_preprocessed_PD.shape

(466285, 118)

In [32]:
df_PD_result=pd.DataFrame({'PD':model_pd_logistic_reg.predict_proba(loan_data_preprocessed_PD)[:][:,0],
                           'index':loan_data_preprocessed_PD.index})

In [33]:
df_PD_result.set_index('index',inplace=True)

In [34]:
df_ExpectedLoss=pd.concat([df_ExpectedLoss,df_PD_result], axis=1)
df_ExpectedLoss.head()

,rcvry_rate_stage_1,rcvry_rate_stage_2,recovery_rate_final,LGD,CCF,EAD,PD
0,1,0.086363,0.086363,0.913637,0.589919,2949.592525,0.165262
1,1,0.084464,0.084464,0.915536,0.777772,1944.429202,0.266145
2,1,0.080391,0.080391,0.919609,0.658296,1579.910501,0.294972
3,1,0.095262,0.095262,0.904738,0.660681,6606.809874,0.206383
4,1,0.088898,0.088898,0.911102,0.708245,2124.736147,0.135269


In [35]:
df_ExpectedLoss['LGD'].describe()

count    466285.000000
mean          0.934413
std           0.057611
min           0.368842
25%           0.882563
50%           0.916653
75%           1.000000
max           1.000000
Name: LGD, dtype: float64

In [36]:
df_ExpectedLoss['ExpectedLoss']=df_ExpectedLoss['PD']*df_ExpectedLoss['LGD']*df_ExpectedLoss['EAD']

In [37]:
df_ExpectedLoss['funded_amnt']=loan_data_preprocessed_LGD_EAD['funded_amnt']

In [38]:
df_ExpectedLoss.head()

,rcvry_rate_stage_1,rcvry_rate_stage_2,recovery_rate_final,LGD,CCF,EAD,PD,ExpectedLoss,funded_amnt
0,1,0.086363,0.086363,0.913637,0.589919,2949.592525,0.165262,445.358275,5000
1,1,0.084464,0.084464,0.915536,0.777772,1944.429202,0.266145,473.790706,2500
2,1,0.080391,0.080391,0.919609,0.658296,1579.910501,0.294972,428.564467,2400
3,1,0.095262,0.095262,0.904738,0.660681,6606.809874,0.206383,1233.639311,10000
4,1,0.088898,0.088898,0.911102,0.708245,2124.736147,0.135269,261.860271,3000


In [39]:
df_ExpectedLoss.describe()

,rcvry_rate_stage_1,rcvry_rate_stage_2,recovery_rate_final,LGD,CCF,EAD,PD,ExpectedLoss,funded_amnt
count,466285.000000,466285.000000,466285.000000,466285.000000,466285.000000,466285.000000,466285.000000,466285.000000,466285.000000
mean,0.601409,0.122789,0.065587,0.934413,0.737579,10814.852575,0.109281,1093.793460,14291.801044
std,0.489609,0.030089,0.057611,0.057611,0.101754,6935.165205,0.071600,1112.254624,8274.371300
min,0.000000,-0.019925,0.000000,0.368842,0.292350,190.367768,0.005579,7.933003,500.000000
25%,0.000000,0.106160,0.000000,0.882563,0.671009,5495.036632,0.055899,356.988841,8000.000000
50%,1.000000,0.126237,0.083347,0.916653,0.730697,9208.455337,0.093483,718.498502,12000.000000
75%,1.000000,0.143158,0.117437,1.000000,0.803447,14692.882979,0.146753,1424.131025,20000.000000
max,1.000000,2.389546,0.631158,1.000000,1.000000,35000.000000,0.719954,13041.387094,35000.000000


# total funded amount by the lender


In [40]:
df_ExpectedLoss['funded_amnt'].sum()

6664052450

# total expected loss

In [41]:
df_ExpectedLoss['ExpectedLoss'].sum()

510019483.6080839

# proportion of expectedLoss

In [42]:
df_ExpectedLoss['ExpectedLoss'].sum()/df_ExpectedLoss['funded_amnt'].sum()

0.07653293359179426